In [1]:
import pandas as pd
import csv
import os
import requests
import urllib3
import csv
import time
import re
import numpy as np
# from random import randint
# from pandas import json_normalize
import censusgeocode as cg

In [2]:
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

In [ ]:
filename = '2022429_Georgia_Daily_VoterBase/Georgia_Daily_VoterBase.txt'

In [ ]:
voter = pd.read_csv(filename, low_memory = False, error_bad_lines=False, sep = '|')

In [ ]:
# cleaning up addresses
voter['RESIDENCE_STATE'] = 'GA'
voter['ZIP5'] =  [x[:5] if pd.isna(x) == False else x for x in voter.RESIDENCE_ZIPCODE ]
voter = voter.fillna('')
# strip whitespaces
cols = voter.select_dtypes(object).columns
voter[cols] = voter[cols].apply(lambda x: x.str.strip())

voter['st_address'] = [x + ' ' + y for (x,y) in zip(voter.RESIDENCE_HOUSE_NUMBER , voter.RESIDENCE_STREET_NAME)]

In [ ]:
addresses = voter.iloc[:,[65,12,63,64]].copy().drop_duplicates()

In [ ]:
# checking out how many of these addresses have already been geocoded
res = ['geocode_results/' + x for x in os.listdir(os.getcwd() + '/geocode_results/')]

In [ ]:
masterresults = pd.DataFrame()
censusresults = pd.DataFrame()
censusapiresults = pd.DataFrame()

In [ ]:
# parse files according to different geocoders
for file in res:
    # R and python outputs files have headers; inherit them
    if (('GeoCode' in file) | ('pythongeocode' in file)):
        df = pd.read_csv(file)
        masterresults = masterresults.append(df)
        continue
    # census website outputs do not have headers; add them
    if 'GeocodeResults' in file:
        try:
            df = pd.read_csv(file, names =  ['id', 'address','match','matchtype','parsed','coordinate','tigerlineid','side'])
            censusresults = censusresults.append(df)
        except Exception:
            print (file)
            continue
    else:
        df = pd.read_csv(file)
        censusapiresults = censusapiresults.append(df)

In [ ]:
old_voter_path = 'Georgia_Daily_VoterBase12.22/Georgia_Daily_VoterBase.txt'

In [ ]:
old_voter = pd.read_csv(old_voter_path, low_memory = False, error_bad_lines=False, sep = '|')

In [ ]:
# cleaning up addresses
old_voter['RESIDENCE_STATE'] = 'GA'
old_voter['ZIP5'] =  [x[:5] if pd.isna(x) == False else x for x in old_voter.RESIDENCE_ZIPCODE ]
old_voter = old_voter.fillna('')
# strip whitespaces
cols = old_voter.select_dtypes(object).columns
old_voter[cols] = old_voter[cols].apply(lambda x: x.str.strip())

old_voter['st_address'] = [x + ' ' + y for (x,y) in zip(old_voter.RESIDENCE_HOUSE_NUMBER , old_voter.RESIDENCE_STREET_NAME)]

In [ ]:
old_voter['onelineaddress'] = [x + ' ' + y + ' ' + z + ' ' + a + ' ' + b for (x,y,z,a,b) in zip(old_voter.RESIDENCE_HOUSE_NUMBER, old_voter.RESIDENCE_STREET_NAME, old_voter.RESIDENCE_CITY, old_voter.RESIDENCE_STATE, old_voter.ZIP5)]

In [ ]:
old_addr = old_voter.iloc[:,[65,12,63,64]].copy().drop_duplicates()

In [ ]:
old_addr.info()

In [ ]:
I did some of the geocoding using batch geocoder and some using the api - treating them separately

In [ ]:
old_address_batch = old_addr.iloc[93137:,:].copy()

In [ ]:
old_address_batch = old_address_batch.reset_index()

In [ ]:
old_addrbatch = np.array_split(old_address_batch, 302)

In [ ]:
old_addrbatch[0].head()

just need to stich up old_address_batch and the results

In [ ]:
censusresults['split'] = [x.split(',') if pd.isna(x) == False else [np.nan,np.nan] for x in censusresults['coordinate']]

In [ ]:
censusresults['lat'] = [x[1] for x in censusresults.split]
censusresults['long'] = [x[0] for x in censusresults.split]

In [ ]:
concatres = pd.concat([masterresults,censusresults])

In [ ]:
concatres = concatres.drop(['coordinate','split'], axis = 1)

In [ ]:
concatres = concatres.drop_duplicates()

In [ ]:
len(concatres.id.unique())

In [ ]:
concalatlong = concatres[['id','lat','long']].drop_duplicates()

In [ ]:
len(concalatlong.id.unique())

In [ ]:
old_address_batch.head()

In [ ]:
old_address_batch1 = pd.merge(old_address_batch, concalatlong, left_on ='index', right_on = 'id', how = 'left')

In [ ]:
old_address_batch1.info()

In [ ]:
old_address_batch.info()

In [ ]:
countres = pd.DataFrame(old_address_batch1.groupby('index')['id'].count())

In [ ]:
countres[countres.id == 2].head()
# these just seem to be on a different level of accuracy, which I guess is ok. 

In [ ]:
# isolating those that have been processed by geocoder - some of them did not return a match
old_address_batch2 = old_address_batch1[pd.isna(old_address_batch1.id) == False].copy()
old_address_batch2 = old_address_batch2.drop(['index','id'], axis = 1)

In [ ]:
old_address_batch1.info()

In [ ]:
old_address_batch2.info()

these are the api results

In [ ]:
old_address_api = old_addr.iloc[:93138,:].copy()

In [ ]:
old_address_api['onelineaddress'] = [x  + ' ' + z + ' ' + a + ' ' + b for (x,z,a,b) in zip(old_address_api.st_address, old_address_api.RESIDENCE_CITY, old_address_api.RESIDENCE_STATE, old_address_api.ZIP5)]

In [ ]:
old_address_api1 = pd.merge(old_address_api,censusapiresults, left_on = 'onelineaddress', right_on = 'input', how = 'left')

In [ ]:
old_address_api2 = old_address_api1[pd.isna(old_address_api1.input) == False].copy()[['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5','coordinates']].copy().drop_duplicates()
old_address_api2['lat'] = [x.split(':')[2].split('}')[0] for x in old_address_api2.coordinates]
old_address_api2['long'] = [x.split(':')[1].split(',')[0] for x in old_address_api2.coordinates]
old_address_api2.lat = old_address_api2.lat.str.strip()
old_address_api2.long = old_address_api2.long.str.strip()
old_address_api2 = old_address_api2.drop('coordinates', axis = 1)

concacenating the two results

In [ ]:
concatresults = pd.concat([old_address_api2,old_address_batch2])

In [ ]:
old_address_batch2.info()

In [ ]:
old_address_api2.info()

In [ ]:
# about 10% of these results were passed thru and not geocoded.
len(concatresults[pd.isna(concatresults.lat) == True])/len(concatresults[pd.isna(concatresults.lat) == False])

In [ ]:
concatresults.head()

In [ ]:
concatresults['processed'] = True

merging the old voter file with the new one

In [ ]:
addrmerge = pd.merge(addresses, concatresults, on = ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5'], how = 'left')